In [1]:
import os
import cv2
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
import matplotlib.pyplot as plt


In [ ]:
# Tham số
dataset_path = "dataset"
image_size = (32, 32)
test_ratio = 0.2
val_ratio = 0.2
batch_size = 64
epochs = 20


def load_data():
    images = []
    labels = []
    folders = os.listdir(dataset_path)
    folders.sort()
    for label in folders:
        folder_path = os.path.join(dataset_path, label)
        for img_name in os.listdir(folder_path):
            img_path = os.path.join(folder_path, img_name)
            img = cv2.imread(img_path)
            if img is None:
                continue
            img = cv2.resize(img, image_size)
            images.append(img)
            labels.append(int(label))  # hoặc int(label.split('_')[0]) nếu dùng tên như "0_stop"
    return np.array(images), np.array(labels), len(folders)

def split_dataset(images, labels, test_ratio=0.2, val_ratio=0.2):
    indices = np.arange(len(images))
    np.random.shuffle(indices)

    total = len(images)
    test_size = int(total * test_ratio)
    val_size = int(total * val_ratio)

    test_idx = indices[:test_size]
    val_idx = indices[test_size:test_size + val_size]
    train_idx = indices[test_size + val_size:]

    X_train, y_train = images[train_idx], labels[train_idx]
    X_val, y_val = images[val_idx], labels[val_idx]
    X_test, y_test = images[test_idx], labels[test_idx]

    return X_train, y_train, X_val, y_val, X_test, y_test



In [ ]:
images, labels, num_classes = load_data()
X_train, y_train, X_val, y_val, X_test, y_test = split_dataset(images, labels)

print("Số lớp:", num_classes)
print("Tập train:", len(X_train))
print("Tập val:", len(X_val))
print("Tập test:", len(X_test))


In [ ]:
class SignDataset(Dataset):
    def __init__(self, X, y, transform=None):
        self.X = X
        self.y = y
        self.transform = transform

    def __getitem__(self, idx):
        img = self.X[idx]
        label = self.y[idx]
        if self.transform:
            img = self.transform(img)
        return img, label

    def __len__(self):
        return len(self.X)

transform = transforms.Compose([
    transforms.ToPILImage(),
    transforms.RandomRotation(10),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor()
])


In [ ]:
train_ds = SignDataset(X_train, y_train, transform)
val_ds = SignDataset(X_val, y_val, transform)
test_ds = SignDataset(X_test, y_test, transform)

train_loader = DataLoader(train_ds, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_ds, batch_size=batch_size)
test_loader = DataLoader(test_ds, batch_size=batch_size)


In [ ]:
class Net(nn.Module):
    def __init__(self, num_classes):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(3, 60, 5)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(60, 30, 5)
        self.fc1 = nn.Linear(30 * 5 * 5, 500)
        self.fc2 = nn.Linear(500, num_classes)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = x.view(-1, 30 * 5 * 5)
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return x


In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = Net(num_classes).to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)


In [ ]:
for epoch in range(epochs):
    model.train()
    running_loss = 0.0
    correct = 0

    for imgs, labels in train_loader:
        imgs, labels = imgs.to(device), labels.to(device)
        optimizer.zero_grad()

        outputs = model(imgs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
        correct += (outputs.argmax(1) == labels).sum().item()

    accuracy = correct / len(train_ds)
    print(f"Epoch {epoch+1}/{epochs}, Loss: {running_loss:.4f}, Accuracy: {accuracy:.4f}")


In [ ]:
torch.save(model.state_dict(), "sign_model.pt")
print("✅ Model đã được lưu thành công.")


In [ ]:
model.eval()
correct = 0

with torch.no_grad():
    for imgs, labels in val_loader:
        imgs, labels = imgs.to(device), labels.to(device)
        outputs = model(imgs)
        predicted = outputs.argmax(1)
        correct += (predicted == labels).sum().item()

val_accuracy = correct / len(val_ds)
print(f"Validation Accuracy: {val_accuracy:.4f}")


In [ ]:
import torch
torch.cuda.empty_cache()
torch.cuda.ipc_collect()
